# Bayes Model-(Abandon)

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import pickle
import seaborn as sns
from collections import Counter
import seaborn as sns

from utils import DataLoader, DataProcesser, label_plot

%load_ext autoreload
%autoreload 2

In [2]:
label_dic = {1: 'Still', 2: 'Walking', 3: 'Run', 4: 'Bike', 5: 'Car', 6: 'Bus', 7: 'Train', 8: 'Subway'}

## Data Preparation

In [5]:
data = DataLoader.SHLDataLoader('data/train/', ratio = None)
data.df = pd.read_csv("data/mid_res/20210525_data_df.csv")
dp = DataProcesser.DataProcesser(data)

Customized DataLoader Passed in.


In [6]:
print("0")
data.df['have_loc'] = data.df.apply(lambda x: 1 if x['accuracy'] > -1 else 0, axis = 1)
data.df['have_gps'] = data.df.apply(lambda x: 1 if x['num_gps'] > 0 else 0, axis = 1)
data.df['have_wifi'] = data.df.apply(lambda x: 1 if x['num_wifi'] > 0 else 0, axis = 1)
data.df['have_cells'] = data.df.apply(lambda x: 1 if x['num_cells'] > 0 else 0, axis = 1)
print("1")
# location
data.df['speed_level'] = pd.cut(data.df['speed'], [float('-inf'), 1, 20, float('inf')], labels = [0, 1, 2]).cat.add_categories(-1).fillna(0).astype('int')
data.df['accuracy_level'] = pd.cut(data.df['accuracy'], [float('-inf'), 5, 10, float('inf')], labels = [0, 1, 2]).cat.add_categories(-1).fillna(0).astype('int')
data.df['speed_wd_max_log'] = np.log(data.df['speed']+1).rolling(20, center = True).max().fillna(method = 'ffill').fillna(method = 'bfill')
data.df['acc_wd_std_log'] = data.df.apply(lambda x: np.log(x['acc_wd_std']*100000 + 1), axis = 1)
print("2")
# gps
data.df['num_gps_level'] = pd.cut(data.df['num_gps'], [0, 5, 10, 15, 50], labels = list(range(4))).cat.add_categories(-1).fillna(0).astype('int')
print("3")
# wifi
data.df['num_wifi_level'] = pd.cut(data.df['num_wifi'], [-1, 1.5, 50], labels = list(range(2))).cat.add_categories(-1).fillna(0).astype('int')
data.df['wifi_freq_5ratio_level'] = pd.cut(data.df['wifi_freq_5ratio'], [round(t, 2) for t in list(np.linspace(-0.01, 1.01, 10))], labels = list(range(9))).cat.add_categories(-1).fillna(0).astype('int')
print("4")
# cells
data.df['num_cells_level'] = pd.cut(data.df['num_cells'], [0, 1.5, 2.5, 3.5, 5.5, 10.5, 50], labels = list(range(6))).cat.add_categories(-1).fillna(0).astype('int')
data.df['cells_isRegistered_mean_level'] = pd.cut(data.df['cells_isRegistered_mean'], [0, 0.2, 0.5, 1.1], labels = list(range(3))).cat.add_categories(-1).fillna(0).astype('int')
data.df['cells_asuLevel_mean_level'] = pd.cut(data.df['cells_asuLevel_mean'], [-1, 15, 100], labels = list(range(2))).cat.add_categories(-1).fillna(0).astype('int')
data.df['cells_dbm_mean'] = data.df.apply(lambda x: x['cells_dbm_mean'] if x['cells_dbm_mean'] < 0 else np.nan, axis = 1)
print("5")

0
1
2
3
4
5


In [7]:
list(data.df)

['time',
 'label',
 'accuracy',
 'latitude',
 'longitude',
 'altitude',
 'num_gps',
 'num_wifi',
 'num_cells',
 'time_dlt',
 'valid_dlt',
 'east',
 'north',
 'east_dlt',
 'north_dlt',
 'east_speed',
 'north_speed',
 'speed',
 'speed_dif',
 'speed_dlt',
 'acc',
 'wifi_rssi_mode',
 'wifi_rssi_mean',
 'wifi_rssi_min',
 'wifi_rssi_max',
 'wifi_rssi_std',
 'wifi_freq_5ratio',
 'cells_ctype_mode',
 'cells_isRegistered_mean',
 'cells_asuLevel_mean',
 'cells_dbm_mean',
 'cells_level_mean',
 'cells_asuLevel_min',
 'cells_dbm_min',
 'cells_level_min',
 'cells_asuLevel_max',
 'cells_dbm_max',
 'cells_level_max',
 'cells_asuLevel_std',
 'cells_dbm_std',
 'gps_snr_mean',
 'gps_snr_min',
 'gps_snr_max',
 'gps_snr_std',
 'speed_log',
 'acc_wd_std',
 'speed_wd_std',
 'have_loc',
 'have_gps',
 'have_wifi',
 'have_cells',
 'speed_level',
 'accuracy_level',
 'speed_wd_max_log',
 'acc_wd_std_log',
 'num_gps_level',
 'num_wifi_level',
 'wifi_freq_5ratio_level',
 'num_cells_level',
 'cells_isRegistered_mean

In [12]:
col_bn = ['time', 'label',
'have_loc', 'have_gps', 'have_wifi', 'have_cells',
'speed_level', 'speed_log', 'speed_wd_max_log', 'accuracy_level', 'acc_wd_std_log',
'num_gps_level', 'gps_snr_max', 'gps_snr_mean',
'num_wifi_level', 'wifi_rssi_mean', 'wifi_rssi_max', 'wifi_freq_5ratio_level',
'num_cells_level', 'cells_asuLevel_mean_level', 'cells_asuLevel_mean', 'cells_isRegistered_mean', 'cells_dbm_mean', 'cells_level_mean']

In [13]:
# 缺失值未处理的版本
data.df[col_bn].to_csv("data/mid_res/20210526_data_bn.csv", header = True, index = False)